In [2]:
import json
import pandas as pd
import requests
import io
from urllib.request import urlopen
import json
import logging
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas.io.json import json_normalize
from pprint import pprint

In [3]:
# load JSON
def load_data(filename):
    df = pd.read_json(filename)
    data_sorted = df['formAnswers'].apply(pd.Series).join(df[['swymerId', 'workDurationInSeconds']])
    return data_sorted.fillna("")

In [4]:
from google.colab import files

#load json files
v1_q = load_data('drive/MyDrive/MA_XAI/answers-v1-basic-group-2-qualification.json')
v1_m = load_data('drive/MyDrive/MA_XAI/answers-v1-basic-group-2-main.json')

v2_q_p1 = load_data('drive/MyDrive/MA_XAI/answers-v2-salient-group-2-qualification.json')
v2_m_p1 = load_data('drive/MyDrive/MA_XAI/answers-v2-salient-group-2-main.json')

v3_q = load_data('drive/MyDrive/MA_XAI/answers-v3-explanations-group-2-qualification.json')
v3_m = load_data('drive/MyDrive/MA_XAI/answers-v3-explanations-group-2-main.json')

# missing data from second group
v2_q_p2 = load_data('drive/MyDrive/MA_XAI/answers-v2-salient-group-2-qualification-part2.json')
v2_m_p2 = load_data('drive/MyDrive/MA_XAI/answers-v2-salient-group-2-main-part2.json')

v2_q = pd.concat([v2_q_p1, v2_q_p2])
v2_m = pd.concat([v2_m_p1, v2_m_p2])

#load journalist
journos = load_data('drive/MyDrive/MA_XAI/answers-journalists_all_versions.json')


In [5]:
#confert to pandas data frame
df_v1_q = pd.DataFrame(v1_q)
df_v1_m = pd.DataFrame(v1_m)

df_v2_q = pd.DataFrame(v2_q)
df_v2_m = pd.DataFrame(v2_m)

df_v3_q = pd.DataFrame(v3_q)
df_v3_m = pd.DataFrame(v3_m)

df_journos = pd.DataFrame(journos)

In [6]:
# remove columns from df_XX_q which are redundant or not needed for the analysis
reduntant_columns = [
  "METADATA.FEATURE",
  "METADATA.GROUP",
  "confirm.0",
]

df_v1_q = df_v1_q.drop(columns=reduntant_columns)
df_v2_q = df_v2_q.drop(columns=reduntant_columns)
df_v3_q = df_v3_q.drop(columns=reduntant_columns)


# get a list of all column names starting with "tutorial."
tutorial_columns = [col for col in df_v1_q.columns if col.startswith("tutorial.")]
tutorial_columns.append("understand-task")

# remove tutorial columns from both dataframes
df_v1_q = df_v1_q.drop(columns=tutorial_columns)
df_v1_m = df_v1_m.drop(columns=tutorial_columns)

df_v2_q = df_v2_q.drop(columns=tutorial_columns)
df_v2_m = df_v2_m.drop(columns=tutorial_columns)

df_v3_q = df_v3_q.drop(columns=tutorial_columns)
df_v3_m = df_v3_m.drop(columns=tutorial_columns)

df_journos = df_journos.drop(columns=tutorial_columns)


# remove "METADATA.PART" columns from both dataframes
df_v1_q = df_v1_q.drop(columns=["METADATA.PART"])
df_v1_m = df_v1_m.drop(columns=["METADATA.PART"])

df_v2_q = df_v2_q.drop(columns=["METADATA.PART"])
df_v2_m = df_v2_m.drop(columns=["METADATA.PART"])

df_v3_q = df_v3_q.drop(columns=["METADATA.PART"])
df_v3_m = df_v3_m.drop(columns=["METADATA.PART"])

df_journos = df_journos.drop(columns=["METADATA.PART"])

In [7]:
# merge df_v1_q and df_v1_m on swymerId,
# only keep rows where aliasId is in both dataframes (inner join)
# keep all columns from both and suffix them with .qualification and .main (e.g. for POINTS)
df_v1 = pd.merge(df_v1_q, df_v1_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))


In [8]:
# dataframe of participants who did the qualification task for v1 and the main task for v2
pd.merge(df_v1_q, df_v2_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))


information-literacy.identify-misinformation  \
0                                             7   
1                                             7   
2                                             6   
3                                             6   
4                                             7   
5                                             3   
6                                             5   
7                                             6   
8                                             5   
9                                             6   
10                                            7   
11                                            4   
12                                            4   
13                                            6   
14                                            6   
15                                            5   
16                                            5   
17                                            6   
18                                            5   
19                                            4   
20                                            4   
21                                            4   
22                                            7   
23                                            2   
24                                            6   
25                                            7   
26                                            6   
27                                            7   
28                                            5   
29                                            7   
30                                            6   
31                                            6   
32                                            4   
33                                            2   
34                                            6   
35                                            4   

   information-seeking.sources.online-newspaper  \
0                                             6   
1                                             6   
2                                             4   
3                                             6   
4                                             6   
5                                             1   
6                                             7   
7                                             5   
8                                             4   
9                                             7   
10                                            5   
11                                            3   
12                                            3   
13                                            6   
14                                            1   
15                                            3   
16                                            2   
17                                            4   
18                                            5   
19                                            1   
20                                            6   
21                                            4   
22                                            7   
23                                            6   
24                                            4   
25                                            1   
26                                            6   
27                                            2   
28                                            2   
29                                            7   
30                                            5   
31                                            7   
32                                            4   
33                                            7   
34                                            6   
35                                            6   

   expectations.sceptical-about-ai  \
0                                7   
1                                5   
2                                3   
3                                5   
4                                5   

In [9]:
# dataframe of participants who are in the main task of v2, but not in either qualification task
df_v2_m[~df_v2_m.swymerId.isin(df_v1_q.swymerId) & ~df_v2_m.swymerId.isin(df_v2_q.swymerId)]

news-dashboard-evaluation.assist-decisions  \
1                                            2   
59                                           3   
99                                           3   
117                                          3   
0                                            7   
39                                           5   

    ai-system-evaluation.most-useful-explanation-feature  \
1                                          readability     
59                             truthfulness-assessment     
99                                         readability     
117                            truthfulness-assessment     
0                              truthfulness-assessment     
39                                     publishing-date     

    newsitem.7.system-evaluation.understand-what-system-does  \
1                                                    5         
59                                                   5         
99                                                   5         
117                                                  3         
0                                                    6         
39                                                   4         

    ai-system-evaluation.easy-to-follow-what-system-does  \
1                                                    4     
59                                                   3     
99                                                   3     
117                                                  3     
0                                                    7     
39                                                   5     

    newsitem.11.system-evaluation.classified-correctly  \
1                                                    2   
59                                                   2   
99                                                   6   
117                                                  5   
0                                                    6   
39                                                   7   

    ai-system-evaluation.tend-to-trust-system-even-without-knowledge  \
1                                                    2                 
59                                                   4                 
99                                                   2                 
117                                                  1                 
0                                                    6                 
39                                                   5                 

    news-dashboard-evaluation.useful-assess-news-articles  \
1                                                    5      
59                                                   3      
99                                                   3      
117                                                  4      
0                                                    6      
39                                                   5      

    newsitem.8.system-evaluation.classified-correctly  \
1                                                   4   
59                                                  3   
99                                                  5   
117                                                 2   
0                                                   6   
39                                                  7   

    newsitem.11.system-evaluation.explanations-comprehensible-and-help-assess  \
1                                                    3                          
59                                                   5                          
99                                                   6                          
117                                                  1                          
0                                                    6                          
39                                                   5                          

    newsitem.8.system-evaluation.explanations-comprehe

In [10]:
# merge df_v2_q and df_v2_m on swymerId,
# only keep rows where aliasId is in both dataframes (inner join)
# keep all columns from both and suffix them with .qualification and .main (e.g. for POINTS)
df_v2 = pd.merge(df_v2_q, df_v2_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))


In [11]:
df_v3 = pd.merge(df_v3_q, df_v3_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))


In [12]:
columns_to_rename = {
  "swymerId": "METADATA.swymerId",
  "workDurationInSeconds.main": "METADATA.workDurationInSeconds.main",
  "workDurationInSeconds.qualification": "METADATA.workDurationInSeconds.qualification",
  "personal-code": "METADATA.personal-code"
}

df_v1 = df_v1.rename(columns=columns_to_rename)
df_v2 = df_v2.rename(columns=columns_to_rename)
df_v3 = df_v3.rename(columns=columns_to_rename)
df_journos = df_journos.rename(columns=columns_to_rename)

In [13]:
# order columns alphabetically
df_v1 = df_v1.reindex(sorted(df_v1.columns), axis=1)
df_v2 = df_v2.reindex(sorted(df_v2.columns), axis=1)
df_v3 = df_v3.reindex(sorted(df_v3.columns), axis=1)
df_journos = df_journos.reindex(sorted(df_journos.columns), axis=1)

In [14]:
from pprint import pprint

columns = list(df_v1.columns)
pprint(columns)

['METADATA.FEATURE',
 'METADATA.GROUP',
 'METADATA.personal-code',
 'METADATA.swymerId',
 'METADATA.workDurationInSeconds.main',
 'METADATA.workDurationInSeconds.qualification',
 'POINTS.main',
 'POINTS.qualification',
 'ai-system-evaluation.additional-functionality',
 'ai-system-evaluation.benefit-of-doubt',
 'ai-system-evaluation.classification-comprehensible',
 'ai-system-evaluation.criteria-to-judge-reliability',
 'ai-system-evaluation.easy-to-follow-what-system-does',
 'ai-system-evaluation.everyday-work-helpful',
 'ai-system-evaluation.know-what-will-happen-next-time',
 'ai-system-evaluation.most-useful-explanation-feature',
 'ai-system-evaluation.other-information-wish',
 'ai-system-evaluation.rely-on-ai-system',
 'ai-system-evaluation.sceptical-about-ai-system',
 'ai-system-evaluation.system-able-to-classify-news-articles',
 'ai-system-evaluation.system-able-to-detect-fake-news',
 'ai-system-evaluation.system-can-correctly-classify-news-articles',
 'ai-system-evaluation.system-

In [15]:
# determine which columns only contain values which can be parsed to numbers
def is_number(s):
    if s == "":
        return True # empty cells are treated as possible numbers
    try:
        float(s)
        return True
    except ValueError:
        return False

In [16]:
# determine which columns only contain values which can be parsed to integers or are empty
# change data type of numeric columns to int
numeric_columns = df_v1.columns[df_v1.applymap(is_number).all()]
df_v1[numeric_columns] = df_v1[numeric_columns].apply(pd.to_numeric, downcast='integer')

numeric_columns = df_v2.columns[df_v2.applymap(is_number).all()]
df_v2[numeric_columns] = df_v2[numeric_columns].apply(pd.to_numeric, downcast='integer')

numeric_columns = df_v3.columns[df_v3.applymap(is_number).all()]
df_v3[numeric_columns] = df_v3[numeric_columns].apply(pd.to_numeric, downcast='integer')

numeric_columns = df_journos.columns[df_journos.applymap(is_number).all()]
df_journos[numeric_columns] = df_journos[numeric_columns].apply(pd.to_numeric, downcast='integer')

In [17]:
rating_columns = [col for col in df_v1.columns if ".rating-" in col]

In [18]:
# replace all value in rating columns which are < 0 with 0
df_v1[rating_columns] = df_v1[rating_columns].mask(df_v1[rating_columns] < 0, 0)
df_v2[rating_columns] = df_v2[rating_columns].mask(df_v2[rating_columns] < 0, 0)
df_v3[rating_columns] = df_v3[rating_columns].mask(df_v3[rating_columns] < 0, 0)
df_journos[rating_columns] = df_journos[rating_columns].mask(df_journos[rating_columns] < 0, 0)

# replace all value in rating columns which are > 100 with 100
df_v1[rating_columns] = df_v1[rating_columns].mask(df_v1[rating_columns] > 100, 100)
df_v2[rating_columns] = df_v2[rating_columns].mask(df_v2[rating_columns] > 100, 100)
df_v3[rating_columns] = df_v3[rating_columns].mask(df_v3[rating_columns] > 100, 100)
df_journos[rating_columns] = df_journos[rating_columns].mask(df_journos[rating_columns] > 100, 100)


In [19]:
#save to excel
df = pd.concat([df_v1, df_v2, df_v3], axis=0, ignore_index=True)
df.to_csv("drive/MyDrive/MA_XAI/df_cleaned.csv", index=False)

df_v1.to_csv("drive/MyDrive/MA_XAI/v1_cleaned_laymen.csv", index=False)
df_v2.to_csv("drive/MyDrive/MA_XAI/v2_cleaned_laymen.csv", index=False)
df_v3.to_csv("drive/MyDrive/MA_XAI/v3_cleaned_laymen.csv", index=False)
df_journos.to_csv("drive/MyDrive/MA_XAI/cleaned_journos.csv", index=False)

#Stats


In [20]:
#df = df[df['POINTS.qualification'] == 2]
#df = df[df["POINTS.main"] >= 5]
#df["METADATA.FEATURE"].value_counts()

In [21]:
df['METADATA.FEATURE'].value_counts()


salient         145
basic           142
explanations    141
Name: METADATA.FEATURE, dtype: int64

In [22]:
#df with cheater
df.shape[0]
df_journos.shape[0]

27

In [23]:
#df = df[df['POINTS.qualification'] == 2]
#df = df[df["POINTS.main"] >= 5]
df_journos["METADATA.FEATURE"].value_counts()
#df.shape[0]

explanations    10
salient         10
basic            7
Name: METADATA.FEATURE, dtype: int64